In [1]:
from IPython.display import display # Allows the use of display() for DataFrames
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

%matplotlib inline
pd.set_option('display.max_columns', None)
sns.set()

In [10]:
demo_df = pd.read_excel("../data/demographics 8-15-16.xlsx")
print demo_df.shape
demo_df.head()

(944, 28)


,PIN,Status,Gender,D.O.B.,Age,Language,Living Set-Up,Medicaid,Hospitalized,RPM Hospitalized,RPM Falls,Rehospitalized,RPM Rehospitalized,CHF,COPD,Diabetes,Hypertension,Stroke,Hypoglycemia,Falls,Liver Disease,Hypotension,Obesity,Coronary Artery Disease,Fractures,Renal,Alzheimer's,Depression
0,10001,A,F,12-22-1940,75.698630,English,Family No CG,N,Y,Y,Y,N,N,NaN,NaN,NaN,X,NaN,X,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN
1,10002,A,F,04-24-1933,83.367123,English,Alone No CG,N,N,NaN,Y,N,NaN,NaN,X,X,X,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN
2,10003,D,F,08-18-1920,96.057534,English,Family No CG,Y,N,NaN,NaN,N,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10004,A,F,12-15-1948,67.712329,English,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10005,D,F,02-18-1934,82.545205,English,NaN,N,N,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
demo_df['Language'].value_counts(dropna=False)

English    634
NaN        162
Spanish    125
Other       22
Female       1
Name: Language, dtype: int64

In [3]:
#Drop duplicate PINS
print demo_df.shape
demo_df.drop_duplicates(['PIN'],keep=False,inplace=True)
print demo_df.shape

demo_df.rename(columns=lambda x: x.lower().replace(" ","_"),inplace=True)
demo_df.rename(columns={"alzheimer's":'alzheimers'},inplace=True)
print demo_df.columns

demo_df['status_active']=demo_df['status'].replace(['M','A','D'],[1,1,0])
demo_df['language']=demo_df['language'].replace(['Female'],['Other'])
demo_df = pd.get_dummies(demo_df,columns=['gender','language'], prefix = ['gender','lang'])
demo_df['d.o.b.']=pd.to_datetime(demo_df['d.o.b.'])
demo_df['living_set-up']=demo_df['living_set-up'].replace(['Alone no CG','Family no CG','Alone with CG'],
                                                          ['Alone No CG','Family No CG','Alone CG'])
demo_df['caregiver']=demo_df['living_set-up'].replace(['Alone No CG','Family No CG','Alone CG','Family CG'],[0,0,1,1])
demo_df['live_alone']=demo_df['living_set-up'].replace(['Alone No CG','Family No CG','Alone CG','Family CG'],[1,0,1,0])

for col in [u'medicaid', u'hospitalized', u'rpm_hospitalized',u'rpm_falls',u'rehospitalized',u'rpm_rehospitalized']:
    demo_df[col]=demo_df[col].replace(['N','Y','Y ',np.nan],[0,1,1,np.nan])

for col in [u'chf', u'copd', u'diabetes',u'hypertension', u'stroke',u'hypoglycemia', u'falls',
            u'liver_disease', u'hypotension',u'obesity', u'coronary_artery_disease',u'fractures',
            u'renal',u'alzheimers', u'depression']:
    demo_df[col]= demo_df[col].replace([np.nan, 'X','x'], [0, 1,1])

demo_df.drop(['living_set-up','status','age'],axis=1,inplace=True)

(944, 28)
(938, 28)
Index([                    u'pin',                  u'status',
                        u'gender',                  u'd.o.b.',
                           u'age',                u'language',
                 u'living_set-up',                u'medicaid',
                  u'hospitalized',        u'rpm_hospitalized',
                     u'rpm_falls',          u'rehospitalized',
            u'rpm_rehospitalized',                     u'chf',
                          u'copd',                u'diabetes',
                  u'hypertension',                  u'stroke',
                  u'hypoglycemia',                   u'falls',
                 u'liver_disease',             u'hypotension',
                       u'obesity', u'coronary_artery_disease',
                     u'fractures',                   u'renal',
                    u'alzheimers',              u'depression'],
      dtype='object')


In [4]:
read_df = pd.read_excel("../data/readings 8-11-16.xlsx",sheetname='Supr&DupsRemoved')
pulseox_df = read_df[read_df['ReadingType']=='PulseOx'].reset_index()
pulseox_df.drop(['index','ReadingId','ReadingType','Systolic','Diastolic','HeartRate','Weight','ReadingSuppressedOn','Steps'],axis=1,inplace=True,errors='ignore')

bp_df = read_df[read_df['ReadingType']=='Blood Pressure'].reset_index()
bp_df.drop(['index','ReadingId','ReadingId','ReadingType','SP02','HeartRate_SP02','Weight','ReadingSuppressedOn','Steps'],axis=1,inplace=True,errors='ignore')

w_df = read_df[read_df['ReadingType']=='Weight'].reset_index()
w_df.drop(['index','ReadingId','ReadingType','Systolic','Diastolic','HeartRate','SP02','HeartRate_SP02','ReadingSuppressedOn','Steps'],axis=1,inplace=True,errors='ignore')

step_df = read_df[read_df['ReadingType']=='Steps'].reset_index()
step_df.drop(['index','ReadingId','ReadingType','Systolic','Diastolic','HeartRate','Weight','SP02','HeartRate_SP02','ReadingSuppressedOn'],axis=1,inplace=True,errors='ignore')

pulseox_df['SPO2_read_date']=pulseox_df['ReadingDate'].apply(lambda x: x.date())
pulseox_df['SPO2_read_time']=pulseox_df['ReadingDate'].apply(lambda x: x.time())
pulseox_df.drop(['ReadingDate'],axis=1,inplace=True,errors='ignore')

bp_df['BPHR_read_date']=bp_df['ReadingDate'].apply(lambda x: x.date())
bp_df['BPHR_read_time']=bp_df['ReadingDate'].apply(lambda x: x.time())
bp_df.drop(['ReadingDate'],axis=1,inplace=True,errors='ignore')

w_df['W_read_date']=w_df['ReadingDate'].apply(lambda x: x.date())
w_df['W_read_time']=w_df['ReadingDate'].apply(lambda x: x.time())
w_df.drop(['ReadingDate'],axis=1,inplace=True,errors='ignore')

step_df['W_read_date']=step_df['ReadingDate'].apply(lambda x: x.date())
step_df['W_read_time']=step_df['ReadingDate'].apply(lambda x: x.time())
step_df.drop(['ReadingDate'],axis=1,inplace=True,errors='ignore')

merge_df = pd.merge(bp_df,w_df,how='inner',left_on=['EndUserId','TIPS ID','BPHR_read_date'],right_on=['EndUserId','TIPS ID','W_read_date'])
print merge_df.shape
merge_df = pd.merge(merge_df,pulseox_df,how='inner',left_on=['EndUserId','TIPS ID','BPHR_read_date'],right_on=['EndUserId','TIPS ID','SPO2_read_date'])
print merge_df.shape
merge_df.insert(1,'measure_date',merge_df['BPHR_read_date'])
merge_df.drop(['W_read_date','BPHR_read_date','SPO2_read_date'],axis=1,inplace=True)
print merge_df.shape

miss_week_f = lambda x: x['measure_date'].diff().fillna(0).div(11).apply(lambda y: pd.tslib.Timedelta(y).days).sum()
total_week_f = lambda x: pd.tslib.Timedelta(x['measure_date'].diff().fillna(0).sum()/7).days

missed_weeks = merge_df[['EndUserId','measure_date']].groupby(['EndUserId']).agg(miss_week_f)
total_weeks = merge_df[['EndUserId','measure_date']].groupby(['EndUserId']).agg(total_week_f)
first_measure_date = merge_df.groupby(['EndUserId'])['measure_date'].min()

compliance_df = pd.merge(missed_weeks.reset_index(),total_weeks.reset_index(),how='inner',on='EndUserId')
#merge_df = pd.merge(merge_df,pulse_df,how='inner',left_on=['EndUserId','BPHR_read_date'],right_on=['EndUserId','SPO2_read_date'])
compliance_df = compliance_df.merge(first_measure_date.reset_index(),how='inner',on='EndUserId')
compliance_df.columns=['EndUserId','missed_wks','total_wks','first_measure_date']
compliance_df['compliance']=1-compliance_df['missed_wks']/compliance_df['total_wks']

initial_measures_df = pd.merge(compliance_df,merge_df,how='left',left_on=['EndUserId','first_measure_date'],right_on=['EndUserId','measure_date'])
initial_measures_df.drop(['BPHR_read_time','W_read_time','measure_date','SPO2_read_time'],axis=1,inplace=True)
print initial_measures_df.shape

data_df = demo_df.merge(initial_measures_df,left_on='pin',right_on='TIPS ID')
data_df['age']=pd.to_datetime(data_df['first_measure_date'])-pd.to_datetime(data_df['d.o.b.'])
data_df.drop(['TIPS ID'],axis=1,inplace=True)
print data_df.shape

(13176, 10)
(12507, 14)
(12507, 12)
(641, 12)
(629, 43)


In [5]:
data_df.head()

,pin,d.o.b.,medicaid,hospitalized,rpm_hospitalized,rpm_falls,rehospitalized,rpm_rehospitalized,chf,copd,diabetes,hypertension,stroke,hypoglycemia,falls,liver_disease,hypotension,obesity,coronary_artery_disease,fractures,renal,alzheimers,depression,status_active,gender_F,gender_M,lang_English,lang_Other,lang_Spanish,caregiver,live_alone,EndUserId,missed_wks,total_wks,first_measure_date,compliance,Systolic,Diastolic,HeartRate,Weight,SP02,HeartRate_SP02,age
0,10001,1940-12-22,0.0,1.0,1.0,1.0,0.0,0.0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,1,0,1,0,0,0.0,0.0,183,12,53,2015-07-13,0.773585,147.0,68.0,55.0,269.2,97.0,57.0,27231 days
1,10002,1933-04-24,0.0,0.0,NaN,1.0,0.0,NaN,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,1,0,0,0.0,1.0,185,24,50,2015-08-10,0.520000,135.0,74.0,69.0,195.8,96.0,62.0,30058 days
2,10004,1948-12-15,0.0,NaN,NaN,NaN,NaN,NaN,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,NaN,NaN,188,1,2,2015-12-21,0.500000,110.0,75.0,72.0,252.0,97.0,71.0,24477 days
3,10006,1948-11-28,0.0,0.0,NaN,NaN,0.0,NaN,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0.0,0.0,193,24,51,2015-07-20,0.529412,122.0,78.0,61.0,250.7,97.0,60.0,24340 days
4,10009,1927-10-06,0.0,0.0,NaN,NaN,0.0,NaN,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0.0,1.0,196,9,53,2015-07-13,0.830189,144.0,86.0,72.0,152.1,93.0,71.0,32057 days


In [9]:
from sets import Set
demo_pins = Set(demo_df['pin'].values)
meas_pins = Set(data_df['pin'].values)
print demo_pins.difference(meas_pins)

Set([10045, 30078, 30039, 60077, 20033, 40025, 220051, 10030, 50011, 20004, 10035, 50034, 60082, 40029, 20018, 20034, 20021, 30059, 20029, 30025, 30133, 20032, 30048, 40002, 20036, 30134, 40006, 30049, 20041, 60087, 50041, 20048, 210001, 210002, 40020, 210005, 210006, 30137, 10041, 210009, 40027, 40028, 210013, 40030, 40031, 210016, 60001, 60002, 40036, 40037, 40017, 60007, 40041, 40042, 60012, 230040, 60021, 30061, 60024, 60025, 230010, 60027, 60029, 230016, 230017, 60034, 60035, 60042, 10042, 230028, 230029, 230030, 60047, 60048, 250001, 60050, 250003, 250004, 250005, 250006, 60055, 250008, 60057, 60058, 60060, 60061, 250014, 60063, 210025, 230050, 60067, 60069, 60070, 60071, 60072, 60073, 60074, 60075, 60076, 230061, 230062, 60080, 60052, 60083, 60084, 60086, 230071, 10053, 200052, 60092, 200053, 60096, 60101, 60102, 60104, 30065, 60106, 230044, 60109, 30098, 200057, 20037, 60124, 220048, 10040, 50043, 60133, 60137, 30099, 60054, 200061, 10024, 310002, 10048, 200062, 30096, 220021, 